In [1]:
import os, sys, ast

import torch

from BIDS import NII, POI
sys.path.append('/home/daniel/Documents/Uni/MT/poi-prediction')
from utils.dataloading_utils import get_gt_pois, compute_surface
from utils.misc import surface_project_coords
import pandas as pd

In [2]:
poi_list = [81, 101, 102, 103, 104, 109, 110, 111, 112, 117, 118, 119, 120, 125, 127, 134, 136, 141, 142, 143, 144, 149, 151]

In [3]:
# Gruber_BGI = BIDS_Global_info(['/home/daniel/Data/Gruber/dataset-poi-gruber'], parents=['rawdata', 'derivatives_seg_new'], additional_key=['seq', 'snapshot', 'ref'])

# def get_files(container):
#     poi_reg = get_gruber_registration_poi(container)
#     poi_gruber = get_gruber_poi(container)
#     vertseg = get_vertseg(container)
#     return poi_reg, poi_gruber, vertseg

# def process_container(subject, container, save_path, rescale_zoom, get_files):
#     poi_reg, poi_gruber, vertseg = get_files(container)
#     # vertseg.reorient_(('L', 'A', 'S'))
#     # poi_reg.reorient_centroids_to_(vertseg)
#     # poi_gruber.reorient_centroids_to_(vertseg)
    
#     vertebrae = set([key[0] for key in poi_reg.keys()])
#     vertseg_arr = vertseg.get_array()

#     summary = []
#     for vert in vertebrae:
#         if vert in vertseg_arr:
#             # x_min, x_max, y_min, y_max, z_min, z_max = get_bounding_box(vertseg_arr, vert)
#             # vertseg_path = os.path.join(save_path, subject, str(vert), 'vertseg.nii.gz')
#             # poi_reg_path = os.path.join(save_path, subject, str(vert), 'poi_reg.json')
#             # gruber_poi_path = os.path.join(save_path, subject, str(vert), 'poi_gruber.json')

#             # if not os.path.exists(os.path.join(save_path, subject, str(vert))):
#             #     os.makedirs(os.path.join(save_path, subject, str(vert)))

#             # vertseg_cropped = vertseg.apply_crop_slice(ex_slice = (slice(x_min, x_max), slice(y_min, y_max), slice(z_min, z_max)))
#             # poi_reg_cropped = poi_reg.crop_centroids(o_shift = (slice(x_min, x_max), slice(y_min, y_max), slice(z_min, z_max)))
#             # poi_gruber_cropped = poi_gruber.crop_centroids(o_shift = (slice(x_min, x_max), slice(y_min, y_max), slice(z_min, z_max)))

#             # if rescale_zoom:
#             #     vertseg_cropped.rescale_(rescale_zoom)
#             #     poi_reg_cropped.rescale_(rescale_zoom)

#             # vertseg_cropped.save(vertseg_path, verbose = False)
#             # poi_reg_cropped.save(poi_reg_path, verbose = False)
#             # poi_gruber_cropped.save(gruber_poi_path, verbose = False)

#             summary.append({   
#                     'subject': subject,
#                     'vertebra': vert,
#                     'file_dir': os.path.join(save_path, subject, str(vert)),
#             })
        
#         else:
#             print(f'Vertebra {vert} has no segmentation for subject {subject}')
        
#     return summary

# master = []
# save_path = '/home/daniel/Data/Gruber/registration_pois'
# rescale_zoom = (1, 1, 1)
# partial_process_container = partial(process_container, save_path = save_path, rescale_zoom = rescale_zoom, get_files = get_files)
# master = pqdm(Gruber_BGI.enumerate_subjects(), partial_process_container, n_jobs = 2, argument_type = 'args', exception_behaviour='immediate')
# master = [item for sublist in master for item in sublist]
# master_df = pd.DataFrame(master)
# master_df.to_csv(os.path.join(save_path, 'master_df.csv'), index = False)
# reg_master = master_df

In [56]:
reg_master = master_df = pd.read_csv('/home/daniel/Data/Gruber/registration_pois/master_df.csv')
gruber_master = pd.read_csv('/home/daniel/Data/Gruber/cutouts_scale-1-1-1/master_df_cleaned.csv')

In [57]:
reg_master.head()

,subject,vertebra,file_dir
0,WS-06,3,/home/daniel/Data/Gruber/registration_pois/WS-...
1,WS-06,4,/home/daniel/Data/Gruber/registration_pois/WS-...
2,WS-06,5,/home/daniel/Data/Gruber/registration_pois/WS-...
3,WS-06,6,/home/daniel/Data/Gruber/registration_pois/WS-...
4,WS-06,7,/home/daniel/Data/Gruber/registration_pois/WS-...


In [58]:
# We only need the columns subject, vertebra, use_sample and bad_poi_list from the Gruber master
gruber_master = gruber_master[['subject', 'vertebra', 'use_sample', 'bad_poi_list']]

In [59]:
#Inner join the two dataframes on the subject and vertebra columns
reg_master_df = reg_master.merge(gruber_master, on = ['subject', 'vertebra'], how = 'inner')

In [60]:
reg_master_df.head()

,subject,vertebra,file_dir,use_sample,bad_poi_list
0,WS-06,3,/home/daniel/Data/Gruber/registration_pois/WS-...,True,[]
1,WS-06,4,/home/daniel/Data/Gruber/registration_pois/WS-...,True,[]
2,WS-06,5,/home/daniel/Data/Gruber/registration_pois/WS-...,True,[]
3,WS-06,6,/home/daniel/Data/Gruber/registration_pois/WS-...,True,[]
4,WS-06,7,/home/daniel/Data/Gruber/registration_pois/WS-...,False,"[104, 151, 127]"


In [61]:
val_subjects = [
                "WS-16",
                "WS-62",
                "WS-45",
                "WS-26"
            ]
test_subjects =[
                "WS-06",
                "WS-55",
                "WS-15",
                "WS-46",
                "WS-48"
            ]

In [62]:
#Keep only the used samples
reg_master_df = reg_master_df[reg_master_df['use_sample'] == True]

In [63]:
#Save for each row the split it belongs to
reg_master_df['split'] = 'train'
reg_master_df.loc[reg_master_df['subject'].isin(val_subjects), 'split'] = 'val'
reg_master_df.loc[reg_master_df['subject'].isin(test_subjects), 'split'] = 'test'

In [64]:
#Drop the train subjects
reg_master_df = reg_master_df[reg_master_df['split'] != 'train']

In [65]:
reg_dist_dict = {
    'subject': [],
    'vertebra': [],
    'poi_idx': [],
    'gruber_poi': [],
    'reg_poi': [],
    'gruber_poi_proj': [],
    'reg_poi_proj': [],
    'gruber_proj_dist': [],
    'reg_proj_dist': [],
    'gruber_reg_dist': [],
    'gruber_reg_proj_dist': [],
    'gruber_proj_reg_proj_dist': [],
    'good_poi': [],
    'split': [],
}

for row_idx in range(len(reg_master_df)):
    row = reg_master_df.iloc[row_idx]
    subject = row['subject']
    vertebra = row['vertebra']
    split = row['split']

    bad_poi_list = ast.literal_eval(row['bad_poi_list'])
    bad_poi_list = [int(poi) for poi in bad_poi_list]
    bad_poi_list = torch.tensor(bad_poi_list)
    
    vertseg_path = os.path.join(row['file_dir'], 'vertseg.nii.gz')

    vertseg_nii = NII.load(vertseg_path, seg = True)
    gruber_poi = POI.load(os.path.join(row['file_dir'], 'poi_gruber.json'))
    reg_poi = POI.load(os.path.join(row['file_dir'], 'poi_reg.json'))

    #Should already be in that format, but it doesn't hurt to make sure
    vertseg_nii.rescale_and_reorient_(axcodes_to=('L', 'A', 'S'), voxel_spacing = (1,1,1), verbose = False)
    gruber_poi.rescale_(voxel_spacing = (1,1,1))
    reg_poi.rescale_(voxel_spacing = (1,1,1))

    vertseg_arr = vertseg_nii.get_array()

    #Get pois in tensor format
    gruber_pois, _ = get_gt_pois(gruber_poi, vertebra, poi_indices = poi_list)
    reg_pois, _ = get_gt_pois(reg_poi, vertebra, poi_indices = poi_list)

    surface = compute_surface(torch.tensor(vertseg_arr.astype(int)))

    gruber_pois_proj, gruber_proj_dist = surface_project_coords(gruber_pois, surface)
    gruber_pois_proj = gruber_pois_proj.squeeze(0)
    gruber_proj_dist = gruber_proj_dist.squeeze(0)
    reg_pois_proj, reg_proj_dist = surface_project_coords(reg_pois, surface)
    reg_pois_proj = reg_pois_proj.squeeze(0)
    reg_proj_dist = reg_proj_dist.squeeze(0)

    for p_idx, gruber_poi, gruber_poi_proj, reg_poi, reg_poi_proj, gruber_dist, reg_dist in zip(poi_list, gruber_pois, gruber_pois_proj, reg_pois, reg_pois_proj, gruber_proj_dist, reg_proj_dist):
        reg_dist_dict['subject'].append(subject)
        reg_dist_dict['vertebra'].append(vertebra)
        reg_dist_dict['poi_idx'].append(p_idx)
        reg_dist_dict['gruber_poi'].append(gruber_poi.numpy())
        reg_dist_dict['reg_poi'].append(reg_poi.numpy())
        reg_dist_dict['gruber_poi_proj'].append(gruber_poi_proj.numpy())
        reg_dist_dict['reg_poi_proj'].append(reg_poi_proj.numpy())
        reg_dist_dict['gruber_proj_dist'].append(gruber_dist.numpy())
        reg_dist_dict['reg_proj_dist'].append(reg_dist.numpy())
        reg_dist_dict['gruber_reg_dist'].append(torch.norm(gruber_poi.float() - reg_poi.float()).numpy())
        reg_dist_dict['gruber_reg_proj_dist'].append(torch.norm(gruber_poi_proj.float() - reg_poi.float()).numpy())
        reg_dist_dict['gruber_proj_reg_proj_dist'].append(torch.norm(gruber_poi_proj.float() - reg_poi_proj.float()).numpy())
        reg_dist_dict['good_poi'].append(p_idx not in bad_poi_list)
        reg_dist_dict['split'].append(split)
        
reg_dist_df = pd.DataFrame(reg_dist_dict)

In [66]:
reg_dist_df.head()

,subject,vertebra,poi_idx,gruber_poi,reg_poi,gruber_poi_proj,reg_poi_proj,gruber_proj_dist,reg_proj_dist,gruber_reg_dist,gruber_reg_proj_dist,gruber_proj_reg_proj_dist,good_poi,split
0,WS-06,3,81,"[34.257, 5.03, 17.694]","[32.723, 6.71, 15.302]","[34, 5, 17]","[33, 7, 17]",0.7406655,1.7447158,3.301094,2.7272758,2.236068,True,test
1,WS-06,3,101,"[33.311, 51.083, 18.711]","[32.719, 50.28, 20.263]","[33, 51, 19]","[33, 50, 20]",0.43258673,0.4759502,1.8449874,1.4807202,1.4142135,True,test
2,WS-06,3,102,"[33.253, 37.279, 24.381]","[33.198, 37.277, 24.892]","[33, 37, 23]","[33, 37, 23]",1.4314368,1.922394,0.5139545,1.922394,0.0,True,test
3,WS-06,3,103,"[33.568, 50.635, 4.095]","[32.534, 50.128, 5.881]","[34, 51, 5]","[33, 51, 6]",1.0671809,0.9958431,2.125089,1.9198184,1.4142135,True,test
4,WS-06,3,104,"[33.377, 33.817, 10.62]","[32.935, 33.645, 10.952]","[33, 36, 11]","[33, 36, 11]",2.247668,2.3563852,0.57893866,2.3563852,0.0,True,test


In [67]:
reg_dist_df = reg_dist_df[['subject', 'vertebra', 'poi_idx', 'gruber_proj_dist', 'reg_proj_dist', 'gruber_reg_dist', 'gruber_reg_proj_dist', 'gruber_proj_reg_proj_dist', 'good_poi', 'split']]

In [68]:
#Only keep good pois
reg_dist_df = reg_dist_df[reg_dist_df['good_poi']]

In [69]:
#Compute mean, median, MSE and Accuracy@2mm for gruber_reg_dist and gruber_proj_reg_proj_dist
metrics_dict = {
    'Split': [],
    'Method': [],
    'Mean': [],
    'Median': [],
    'MSE': [],
    'Accuracy': [],
}

for split in ['val', 'test']:
    split_df = reg_dist_df[reg_dist_df['split'] == split]
    for metric in ['gruber_reg_dist', 'gruber_proj_reg_proj_dist']:
        mean = split_df[metric].mean()
        median = split_df[metric].median()
        MSE = ((split_df[metric] ** 2).mean()) ** 0.5
        accuracy = (split_df[metric] <= 2).mean()

        metrics_dict['Method'].append(metric)
        metrics_dict['Mean'].append(mean)
        metrics_dict['Median'].append(median)
        metrics_dict['MSE'].append(MSE)
        metrics_dict['Accuracy'].append(accuracy)
        metrics_dict['Split'].append(split)


metrics_df = pd.DataFrame(metrics_dict)

print(metrics_df.to_latex(index = False, float_format = "{:0.2f}".format))

\begin{tabular}{llrrrr}
\toprule
Split & Method & Mean & Median & MSE & Accuracy \\
\midrule
val & gruber_reg_dist & 9.50 & 2.85 & 63.50 & 0.31 \\
val & gruber_proj_reg_proj_dist & 3.86 & 2.45 & 7.11 & 0.37 \\
test & gruber_reg_dist & 3.52 & 2.96 & 4.19 & 0.27 \\
test & gruber_proj_reg_proj_dist & 3.00 & 2.45 & 3.67 & 0.36 \\
\bottomrule
\end{tabular}



In [70]:
#Drop outliers (> 95 percentile)
reg_dist_df_no_outliers = reg_dist_df.copy()

threshold = 0.95
upper_bound = reg_dist_df_no_outliers['reg_proj_dist'].quantile(threshold)

reg_dist_df_no_outliers = reg_dist_df_no_outliers[(reg_dist_df_no_outliers['reg_proj_dist'] <= upper_bound)]

In [71]:
#Compute metrics again on the cleaned dataset
metrics_dict_no_outliers = {
    'Split': [],
    'Method': [],
    'Mean': [],
    'Median': [],
    'MSE': [],
    'Accuracy': [],
}

for split in ['val', 'test']:
    split_df = reg_dist_df_no_outliers[reg_dist_df_no_outliers['split'] == split]
    for metric in ['gruber_reg_dist', 'gruber_proj_reg_proj_dist']:
        mean = split_df[metric].mean()
        median = split_df[metric].median()
        MSE = ((split_df[metric] ** 2).mean()) ** 0.5
        accuracy = (split_df[metric] <= 2).mean()

        metrics_dict_no_outliers['Method'].append(metric)
        metrics_dict_no_outliers['Mean'].append(mean)
        metrics_dict_no_outliers['Median'].append(median)
        metrics_dict_no_outliers['MSE'].append(MSE)
        metrics_dict_no_outliers['Accuracy'].append(accuracy)
        metrics_dict_no_outliers['Split'].append(split)


metrics_df_no_outliers = pd.DataFrame(metrics_dict_no_outliers)

print(metrics_df_no_outliers.to_latex(index = False, float_format = "{:0.2f}".format))

\begin{tabular}{llrrrr}
\toprule
Split & Method & Mean & Median & MSE & Accuracy \\
\midrule
val & gruber_reg_dist & 3.30 & 2.69 & 4.27 & 0.33 \\
val & gruber_proj_reg_proj_dist & 3.04 & 2.45 & 4.15 & 0.38 \\
test & gruber_reg_dist & 3.30 & 2.87 & 3.84 & 0.28 \\
test & gruber_proj_reg_proj_dist & 2.96 & 2.45 & 3.61 & 0.36 \\
\bottomrule
\end{tabular}



In [21]:
#Do the same for the test set
test_reg_dist_df = reg_dist_df[reg_dist_df['split'] == 'test']

#Compute mean, median, MSE and Accuracy@2mm for gruber_reg_dist and gruber_proj_reg_proj_dist
metrics_dict = {
    'metric': [],
    'mean': [],
    'median': [],
    'MSE': [],
    'Accuracy@2mm': [],
    'split': [],
}

for metric in ['gruber_reg_dist', 'gruber_proj_reg_proj_dist']:
    for split in ['test']:
        split_df = test_reg_dist_df[test_reg_dist_df['split'] == split]
        mean = split_df[metric].mean()
        median = split_df[metric].median()
        MSE = ((split_df[metric] ** 2).mean()) ** 0.5
        accuracy = (split_df[metric] <= 2).mean()
        
        metrics_dict['metric'].append(metric)
        metrics_dict['mean'].append(mean)
        metrics_dict['median'].append(median)
        metrics_dict['MSE'].append(MSE)
        metrics_dict['Accuracy@2mm'].append(accuracy)
        metrics_dict['split'].append(split)

metrics_df = pd.DataFrame(metrics_dict)

print(metrics_df.to_latex(index = False))

\begin{tabular}{lrrrrl}
\toprule
metric & mean & median & MSE & Accuracy@2mm & split \\
\midrule
gruber_reg_dist & 3.683175 & 3.138101 & 4.396411 & 0.247075 & test \\
gruber_proj_reg_proj_dist & 3.151477 & 2.449490 & 3.875399 & 0.335517 & test \\
\bottomrule
\end{tabular}

